<a href="https://colab.research.google.com/github/KTH-EXPECA/examples/blob/main/public_container_at_ep5g.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [ ]:
import os, re
from getpass import getpass

with open('demo_project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

Install required packages and dependencies. Ignore the warnings.

In [ ]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

Import packages

In [ ]:
import json
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, show_reservation_byname, get_available_publicips, get_segment_ids, get_worker_interfaces

In the next cell, we reserve the required equipment and resources to form an end to end experiment setup. We reserve EP5G network, one Advantech router and one worker to run the workloads.

In [ ]:
# worker reservation
worker_name = "worker-02"
worker_lease = show_reservation_byname(worker_name + "-lease")
if not worker_lease:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
    )
worker_reservation_id = worker_lease["reservations"][0]["id"]

# ep5g reservation
ep5g_lease = show_reservation_byname("ep5g-lease")
segment_ids = get_segment_ids('ep5g')
if not ep5g_lease:
    ep5g_lease = reserve(
        { "type":"network", "name": "ep5g", "net_name": "ep5g-vip", "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

In the following section we setup the networking equipment for ep5g (refer to [here](https://kth-expeca.gitbook.io/testbedconfig/enroll/enroll-network-segments/ep5g) for more info). It contains creation of an edge-net, a router, and some interfaces on the router and routes.

In [ ]:
# create edge-net
edgenet = chi.network.create_network("edge-net")
chi.network.create_subnet("edge-net-subnet",edgenet["id"],"10.70.70.0/24",gateway_ip="10.70.70.1",enable_dhcp=False)
logger.success("edge-net is created.")

# create ep5g-vip-router
router = chi.network.create_router("ep5g-vip-router","public")
logger.success("ep5g-vip-router router is created.")
logger.info(f"{json.dumps(router,indent=4)}")

# connect ep5g-vip-net to ep5g-vip-router
ep5gnet = chi.network.get_network("ep5g-vip-net")
portadd = chi.network.add_subnet_to_router(router["id"],ep5gnet["subnets"][0])
logger.success("An interface on ep5g-vip-net is added to the router")

# create edge-net to ep5g-vip-router
edgenet = chi.network.get_network("edge-net")
portadd = chi.network.add_subnet_to_router(router["id"],edgenet["subnets"][0])
logger.success("An interface on edge-net is added to the router")

# add ep5g route to ep5g-vip-router
routeadd = chi.network.add_route_to_router(router["id"],"172.16.0.0/16","10.30.111.10")
logger.success("the route added to the router")

Now we start running the container.

In [ ]:
# check available interfaces of the worker
interfaces = list(get_worker_interfaces(worker_name).values())[0]
available_ifs = []
for interface in interfaces.keys():
  if len(interfaces[interface]['connections']) == 0:
    available_ifs.append(interface)
logger.info(f"Available interfaces on {worker_name}: {available_ifs}")
if len(available_ifs) < 2:
  logger.info(f"{json.dumps(interfaces, indent=4)}")
  raise Exception(f"Did not find enough interfaces on {worker_name}")

# run container
edgenet = chi.network.get_network("edge-net")
container_name = "openrtist-server"
chi.container.create_container(
    name = container_name,
    image = "cmusatyalab/openrtist:stable",
    reservation_id = worker_reservation_id,
    mounts = [],
    nets = [
        { "network" : edgenet['id'] },
    ],
    labels = {
        "networks.1.interface":available_ifs[0],
        "networks.1.ip":"10.70.70.30/24",
        "networks.1.routes":"172.16.0.0/16-10.70.70.1",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container")